<a href="https://colab.research.google.com/github/MittalMonika/DataScience/blob/master/yolo_Sign_permodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
%cd /content/drive/MyDrive/
import os

/content/drive/MyDrive


In [ ]:
# copy content to install_required_packages.py
# Run this script to install required packages
!pip install requests PyPDF2 pytesseract pdf2image
!sudo apt install tesseract-ocr
!apt-get install poppler-utils
!pip install ultralytics

In [15]:
# main.py
import os
import requests
from pdf2image import convert_from_path
from PIL import Image, ImageDraw
import random
import shutil


In [16]:
# Function to create directories
def create_directories(base_path, sub_paths):
    for path in sub_paths:
        full_path = os.path.join(base_path, path)
        os.makedirs(full_path, exist_ok=True)
        print(f"Directory created: {full_path}")


In [17]:
# Function to download PDFs
def download_pdfs(base_url, form_names, download_path):
    for form in form_names:
        pdf_url = f"{base_url}/{form}.pdf"
        file_path = os.path.join(download_path, f"{form}.pdf")
        if not os.path.exists(file_path):
            response = requests.get(pdf_url)
            if response.status_code == 200:
                with open(file_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded: {form}.pdf")
            else:
                print(f"Failed to download {form}.pdf")

In [18]:
# Function to convert PDF to images
def convert_pdf_to_images(pdf_path, output_dir, form_n):
    images = convert_from_path(pdf_path)
    for i, image in enumerate(images):
        image_path = os.path.join(output_dir, f"{form_n}_page_{i + 1}.png")
        image.save(image_path, "PNG")
        print(f"Saved image: {image_path}")

In [19]:
def crop_annotation(image_path, annotation):
    # Load the image
    image = Image.open(image_path)
    img_width, img_height = image.size

    # Parse the annotation
    _, x_center, y_center, width, height = map(float, annotation.split())

    # Convert YOLO format to pixel values
    box_width = width * img_width
    box_height = height * img_height
    x_center *= img_width
    y_center *= img_height

    # Calculate top-left corner of the bounding box
    x1 = int(x_center - box_width / 2)
    y1 = int(y_center - box_height / 2)

    # Crop the image
    cropped_image = image.crop((x1, y1, x1 + int(box_width), y1 + int(box_height)))

    return cropped_image

In [27]:
def paste_on_background(background_path, cropped_image, output_image_path,output_box_path, position):
    background = Image.open(background_path)
    background_width, background_height = background.size

    # Create a copy of the background for drawing the bounding box
    background_with_box = background.copy()


    # Paste the cropped image onto the background at the specified position
    background.paste(cropped_image, position, cropped_image if cropped_image.mode == 'RGBA' else None)
    background_with_box.paste(cropped_image, position, cropped_image if cropped_image.mode == 'RGBA' else None)


    # Draw bounding box for visualization (optional)
    draw = ImageDraw.Draw(background_with_box)
    x1, y1 = position
    x2, y2 = x1 + cropped_image.width, y1 + cropped_image.height
    draw.rectangle([x1, y1, x2, y2], outline="red")

    # Save the result
    background.save(output_image_path)
    background_with_box.save(output_box_path.replace('.png', '_bbox.png'))
    print(f"Saved images to {output_image_path} and {output_box_path.replace('.png', '_bbox.png')}")


    # Calculate YOLO format coordinates
    bbox_x_center = (x1 + x2) / 2 / background_width
    bbox_y_center = (y1 + y2) / 2 / background_height
    bbox_width = cropped_image.width / background_width
    bbox_height = cropped_image.height / background_height

    yolo_format = (bbox_x_center, bbox_y_center, bbox_width, bbox_height)
    return yolo_format

In [24]:
def process_images(output_dir, ipath, bpath, lpath, form_names):

    # COMMENT :  need to automate this automatically takes the labesl from the file
    # # give the co-ordinates of the sign box here with name of the

    specialvar = form_names[0]
    exclusion_pattern = specialvar + "_page_1.png"
    document_paths = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('_page_3.png') and f != exclusion_pattern]


    label_image = "0 0.506875 0.77578125 0.9265625 0.0898125"
    file_path = os.path.join(output_dir, specialvar + '_page_1.png')
    cropped_image = crop_annotation(file_path, label_image)
    cropped_image.save(os.path.join(bpath, 'cropped_image_' + specialvar + '.png'))

    for doc_path in document_paths:
        for i in range(20):
            pos_y = random.randint(40, 2000)
            position = (100, pos_y)
            base_name = os.path.basename(doc_path).replace('.png', '')
            output_image_path = os.path.join(ipath, f"{base_name}_{specialvar}_{i}.png")
            output_box_path = os.path.join(bpath, f"{base_name}_{specialvar}_{i}_bbox.png")

            yolo_coordinates = paste_on_background(doc_path, cropped_image, output_image_path, output_box_path, position)

            with open(os.path.join(lpath, f"{base_name}_{specialvar}_{i}.txt"), 'w') as file:
                file.write(f"0 {yolo_coordinates[0]} {yolo_coordinates[1]} {yolo_coordinates[2]} {yolo_coordinates[3]}")


In [32]:
def copy_images_and_create_labels(output_dir, ipath, lpath):
    document_paths = [os.path.join(output_dir, f) for f in os.listdir(output_dir)]

    for doc_path in document_paths:
        # Extract the base file name without page number and extension
        base_file_name = os.path.basename(doc_path).rsplit('_', 2)[0]

        page_num = 2  # Starting from the second page
        while True:
            # Construct the new page path
            page_path = os.path.join(os.path.dirname(doc_path), f"{base_file_name}_page_{page_num}.png")
            if not os.path.exists(page_path):
                break  # Stop if the page does not exist

            # Copy image to training images directory
            dest_image_path = os.path.join(ipath, f"{base_file_name}_page_{page_num}.png")
            shutil.copyfile(page_path, dest_image_path)

            # Create an empty file in labels directory
            with open(os.path.join(lpath, f"{base_file_name}_page_{page_num}.txt"), 'w') as file:
                pass  # Empty file

            page_num += 1

    print("Images and labels have been successfully copied and created.")


In [34]:
def prepare_validation_set(ipath, ivpath, lpath, lvpath, validation_ratio=0.2):
    # Get a list of filenames in the training images directory
    train_filenames = [os.path.splitext(f)[0] for f in os.listdir(ipath) if f.endswith('.png')]

    # Calculate the number of images to move to validation
    num_valid = int(len(train_filenames) * validation_ratio)

    # Randomly select images for validation
    valid_filenames = random.sample(train_filenames, num_valid)

    # Copy selected images and their corresponding label files to validation directories
    for filename in valid_filenames:
        # Copy image
        src_image_path = os.path.join(ipath, filename + '.png')
        dst_image_path = os.path.join(ivpath, filename + '.png')
        shutil.copyfile(src_image_path, dst_image_path)

        # Copy label file
        src_label_path = os.path.join(lpath, filename + '.txt')
        dst_label_path = os.path.join(lvpath, filename + '.txt')
        shutil.copyfile(src_label_path, dst_label_path)

    print(f"Copied {num_valid} images and their labels to the validation directories.")


In [36]:
def copy_specific_files_to_test(output_dir, itestpath, file_pattern):
    for filename in os.listdir(output_dir):
        if filename.endswith(file_pattern):
            source_file_path = os.path.join(output_dir, filename)
            destination_file_path = os.path.join(itestpath, filename)
            shutil.copyfile(source_file_path, destination_file_path)
            print(f"Copied {filename} to {itestpath}")

In [54]:
import subprocess

def train_yolo(model_path, data_yaml, epochs, img_size):
    command = f"yolo task=detect mode=train model={model_path} data={data_yaml} epochs={epochs} imgsz={img_size}"
    # to supress all the outputs on scree
    #subprocess.run(command, shell=True)
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # Print output in real-time
    for line in iter(process.stdout.readline, b''):
        print(line.decode().rstrip())

    process.wait()

def validate_yolo(model_path, data_yaml):
    command = f"yolo task=detect mode=val model={model_path} data={data_yaml}"
    #subprocess.run(command, shell=True)
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # Print output in real-time
    for line in iter(process.stdout.readline, b''):
        print(line.decode().rstrip())

    process.wait()

def predict_yolo(model_path, source, output_dir, confidence):
    command = f"yolo task=detect mode=predict model={model_path} conf={confidence} source={source} name={output_dir} save_txt=True"
    #subprocess.run(command, shell=True)
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # Print output in real-time
    for line in iter(process.stdout.readline, b''):
        print(line.decode().rstrip())

    process.wait()


In [ ]:
#Main function
def main():
    # Define base and download paths
    base_path = os.path.join(os.getcwd(), "yoloSign1")
    download_path = os.path.join(base_path, "Pdf_document")
    output_dir = os.path.join(download_path, "Pdf_To_Images")

    # Create directories
    sub_paths = ["Pdf_document", "Pdf_document/Pdf_To_Images", "train/images", "train/labels", "train/box_images", "test/images", "valid/images", "valid/labels","detect"]
    create_directories(base_path, sub_paths)


    tpath = base_path + "/train"
    ipath =  tpath + "/images/"
    lpath  = tpath + "/labels/"
    bpath  =  tpath + "/box_images/"
    testpath = base_path + "/test"
    itestpath = testpath + "/images/"
    vpath = base_path + "/valid"
    ivpath = vpath + "/images/"
    lvpath = vpath + "/labels/"
    inf_dir =  base_path + "/detect"

    # Define form names and URL
    form_names = ['fw4', 'f3115', 'f1120ric', 'f1120pc', 'f1120', 'f8453']
    base_url = "https://www.irs.gov/pub/irs-pdf"

    # Download PDFs
    download_pdfs(base_url, form_names, download_path)

    # Convert downloaded PDFs to images
    for form in form_names:
          pdf_path = os.path.join(download_path, f"{form}.pdf")
          convert_pdf_to_images(pdf_path, output_dir, form)

    # Process images
    process_images(output_dir, ipath, bpath, lpath, form_names)

    # Copy images and create label files
    copy_images_and_create_labels(output_dir, ipath, lpath)


    # Prepare validation set
    prepare_validation_set(ipath, ivpath, lpath, lvpath)


    #Prepare test data
    file_pattern = 'fw4_page_1.png'  # Define the specific file pattern to be copied
    copy_specific_files_to_test(output_dir, itestpath, file_pattern)



    ## Paths for YOLO model and data

    # COMMENT :  need to think of how to create these data_yaml automatically
    model_dir_train = base_path+"yolo8_best.pt"
    data_yaml = base_path+"/data.yaml"

    # Train YOLO model
    train_yolo(model_dir_train, data_yaml, epochs=1, img_size=640)


     # COMMENT : need to automate the output of yolo model file into a known place so that i can automate the path

    # Validate model

    model_dir = "runs/detect/train9/weights/best.pt"
    validate_yolo(model_dir, data_yaml)

    # Predict with model

    predict_yolo(model_dir, itestpath, inf_dir, confidence=0.25)


if __name__ == "__main__":
    main()